## Data Offers Heatmap

In [1]:
import itertools
import datetime as dt
import psycopg2
import pandas as pd
import plotly
import plotly.plotly as py
import plotly.graph_objs as go

from nbstyler import DATA_STYLE as DS

plotly.offline.init_notebook_mode(connected=True)

%matplotlib notebook
%matplotlib inline

### Objectives

The objective of this notebook recipe is to demonstrate how to compose and style a heatmap (a graphical representation of data where the individual values contained in a matrix are represented as colors). In this case the individual values consist of the total number of business intelligence and data analytics job offers published on a large employment web portal. 

### Data Preparation

Get the data and prepare the `Pandas.DataFrame` object.

In [2]:
conn = psycopg2.connect('dbname=jobsbg')
daily_df = pd.read_sql_query('SELECT * FROM v_data_offers_count_daily', conn, index_col='subm_date')
conn.close()

daily_df.index = pd.to_datetime(daily_df.index)
daily_df.head(5)

,subm_count
subm_date,
2017-09-27,4
2017-09-28,4
2017-09-29,2
2017-09-30,0
2017-10-01,0


#### Shaping the dataframe

Since I wanted to shape the heatmap in a way where the most active day of the week (Monday) is on the bottom, and the other days of the week following on top, I have to provide `Plotly` with a list of list with each of the lists being the same day (e.g. Monday) for the whole time period. In order to do that I first filled in the missing dates to create a complete rectangular dataframe table which could then be reshaped. See also: https://stackoverflow.com/a/45850005_

In [3]:
min_ts = min(daily_df.index)
max_ts = max(daily_df.index)
idx = pd.date_range(
    min_ts - dt.timedelta(days=min_ts.weekday()),
    max_ts + dt.timedelta(days=6-max_ts.weekday()))

In [4]:
daily_df = daily_df.reindex(idx)
daily_df.head()

,subm_count
2017-09-25,NaN
2017-09-26,NaN
2017-09-27,4.0
2017-09-28,4.0
2017-09-29,2.0


Now the dataframe can be reshaped into columns representing the days of the week, and then transposed to completely match our heatmap structure. That will make composing the chart itself very easy.

In [15]:
matrix_df = pd.DataFrame(daily_df.values.reshape(len(daily_df)//7, 7), columns=daily_df.index[:7].strftime('%A'))
matrix_df = matrix_df.T
matrix_df.head(7)

,0,1,2,3,4,5,6,7,8,9,...,77,78,79,80,81,82,83,84,85,86
Monday,NaN,5.0,4.0,4.0,2.0,7.0,5.0,4.0,3.0,6.0,...,9.0,8.0,8.0,13.0,12.0,12.0,0.0,0.0,14.0,15.0
Tuesday,NaN,12.0,4.0,4.0,9.0,2.0,6.0,5.0,5.0,7.0,...,6.0,4.0,4.0,5.0,3.0,6.0,17.0,12.0,8.0,6.0
Wednesday,4.0,2.0,4.0,6.0,5.0,1.0,5.0,4.0,2.0,6.0,...,5.0,3.0,12.0,6.0,9.0,3.0,0.0,9.0,15.0,8.0
Thursday,4.0,7.0,2.0,4.0,4.0,3.0,3.0,2.0,2.0,2.0,...,4.0,16.0,9.0,6.0,4.0,13.0,6.0,11.0,6.0,11.0
Friday,2.0,7.0,1.0,8.0,5.0,3.0,4.0,5.0,9.0,9.0,...,8.0,10.0,9.0,13.0,11.0,0.0,11.0,14.0,11.0,0.0
Saturday,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
Sunday,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN


#### Building the heatmap

The heatmap is composed of three dimensions: x, y, and z. The z dimension will hold our values matrix. Let's prepare that first:

In [6]:
vals = [
    [v for v in matrix_df.loc['Monday'].values],
    [v for v in matrix_df.loc['Tuesday'].values],
    [v for v in matrix_df.loc['Wednesday'].values],
    [v for v in matrix_df.loc['Thursday'].values],
    [v for v in matrix_df.loc['Friday'].values],
    [v for v in matrix_df.loc['Saturday'].values],
    [v for v in matrix_df.loc['Sunday'].values]
]

A manually defined color scale to match with the presentation style:

In [7]:
colorscale =[
    [0.0, DS['colorramp']['acc1'][0]],
    [0.1111111111111111, DS['colorramp']['acc1'][1]],
    [0.2222222222222222, DS['colorramp']['acc1'][2]],
    [0.3333333333333333, DS['colorramp']['acc1'][3]],
    [0.4444444444444444, DS['colorramp']['acc1'][4]],
    [0.5555555555555556, DS['colorramp']['acc1'][5]],
    [0.6666666666666666, DS['colorramp']['acc1'][6]],
    [0.7777777777777778, DS['colorramp']['acc1'][7]],
    [0.8888888888888888, DS['colorramp']['acc1'][8]],
    [1.0, DS['colorramp']['acc1'][9]]
]

Hover labels to display when hovering individual cells:

In [8]:
z_dim_labels = [v for v in matrix_df.values]

Finally, the heatmap trace definition:

In [9]:
hm_trace = go.Heatmap(
    x=[w for w in daily_df.index[::7]],
    y=[d for d in matrix_df.index],
    z=vals,
    hoverinfo='text',
    text=z_dim_labels,
    colorscale=colorscale,
    showscale=False,
    xgap=0,
    ygap=0,
) 

data = [hm_trace]

Layout preparation is fairly standard. Labels have been turned off for both axes.

In [10]:
layout = go.Layout(
    paper_bgcolor=DS['colors']['bg1'],
    plot_bgcolor=DS['colors']['bg1'],
    title='Data Jobs Daily Heatmap',
    titlefont=DS['chart_fonts']['title'],
    font=DS['chart_fonts']['text'],
    autosize=True,
    showlegend=False,
    hidesources=True,
    yaxis=dict(),
)

In [11]:
fig = go.Figure(data=data, layout=layout)

plotly.offline.iplot(fig, filename='data_jobs_subm_heatmap.html')

In [14]:
# Uncomment the line below to export an HTML version of the chart.
#plotly.offline.plot(fig, filename='data_offers_subm_heatmap.html', show_link=False)

'file:///data/WORKSPACE/jpynb_Employment_Trends_Bulgaria/workbooks/data_offers_subm_heatmap.html'

In [13]:
from IPython.core.display import HTML
with open('../resources/styles/datum.css', 'r') as f:
    style = f.read()
HTML(style)